In [6]:
# !pip freeze --requirement requirements.txt > requirements-freeze.txt
# !pip install -r requirements.txt

In [26]:
# importar bibliotecas de ciencias de dados
import pandas as pd
import numpy as np
import math
import statistics
from scipy import stats
from statsmodels.stats.proportion import proportion_confint
SEED = 42

In [8]:
# ler o arquivo csv agpop
df = pd.read_csv('agpop.csv', sep=',', index_col='Unnamed: 0')
df.head()

,county,state,acres92,acres87,acres82,farms92,farms87,farms82,largef92,largef87,largef82,smallf92,smallf87,smallf82,region
1,ALEUTIAN ISLANDS AREA,AK,683533.0,726596.0,764514.0,26,27,28,14,16,20,6,4,1,W
2,ANCHORAGE AREA,AK,47146.0,59297.0,256709.0,217,245,223,9,10,11,41,52,38,W
3,FAIRBANKS AREA,AK,141338.0,154913.0,204568.0,168,175,170,25,28,21,12,18,25,W
4,JUNEAU AREA,AK,210.0,214.0,127.0,8,8,12,0,0,0,5,4,8,W
5,KENAI PENINSULA AREA,AK,50810.0,85712.0,98035.0,93,119,137,9,18,17,12,18,19,W


Informações dos anos de 1982, 1987 e 1992 sobre:
- acres
- número de fazendas
- fazendas pequenas
- fazendas grandes
- região

Separada por cidade e estado.

Format
This data frame contains the following columns:

county:
county name (character variable)

state:
state abbreviation (character variable)

acres92:
number of acres devoted to farms, 1992

acres87:
number of acres devoted to farms, 1987

acres82:
number of acres devoted to farms, 1982

farms92:
number of farms, 1992

farms87:
number of farms, 1987

farms82:
number of farms, 1982

largef92:
number of farms with 1,000 acres or more, 1992

largef87:
number of farms with 1,000 acres or more, 1987

largef82:
number of farms with 1,000 acres or more, 1982

smallf92:
number of farms with 9 acres or fewer, 1992

smallf87:
number of farms with 9 acres or fewer, 1987

smallf82:
number of farms with 9 acres or fewer, 1982

region:
S = south; W = west; NC = north central; NE = northeast

## Análise e Tratamento

In [9]:
def print_is_duplicated(df):
    """
    Função que verifica se os dados estão duplicados. Se sim,
    mostra a quantidade de dados duplicados.
    """
    confirmation = df.duplicated().values.any()
    print(f"Existem valores duplicados? {confirmation}")
    if confirmation:
        print(f"Existem no total {df.duplicated().sum()} linhas duplicadas.")
    else:
        print('Não existem linhas duplicadas.')


def print_is_na(df):
    """
    Função que verifica se os dados estão duplicados. Se sim,
    retorna um dataframe com a quantidade de dados duplicados
    em valor absoluto e a porcentagem relativa.
    """
    confirmation = df.isnull().values.any()
    print(f"Existem valores nulos? {confirmation}\n")
    if confirmation:
        na = pd.concat([df.isna().sum(), round((df.isna().sum()/df.shape[0])*100, 2)], axis = 1).reset_index()
        na.columns = ["coluna", "absoluto", "porcentagem"]
#         print(na )
        return na
    else:
        print('Não existem valores ausentes.')

In [10]:
# ver se existem valores duplicados
print_is_duplicated(df)

Existem valores duplicados? False
Não existem linhas duplicadas.


In [11]:
# ver se existem valores nulos
df_na = print_is_na(df)
df_na

Existem valores nulos? True



,coluna,absoluto,porcentagem
0,county,0,0.00
1,state,0,0.00
2,acres92,19,0.62
3,acres87,23,0.75
4,acres82,17,0.55
5,farms92,0,0.00
6,farms87,0,0.00
7,farms82,0,0.00
8,largef92,0,0.00
9,largef87,0,0.00


In [12]:
# pegar df_na onde porcentagem é maior que 0
df_na = df_na[df_na['porcentagem'] > 0]

In [13]:
from scipy.stats import shapiro

df_normal = pd.DataFrame({
    'columns': df_na['coluna']
})

normal_list = []
for coluna in df_na['coluna']:
    try:
        _, p_value = shapiro(df[coluna])
        normal_list.append('Normal' if p_value > 0.05 else 'Não normal')
    except:
        normal_list.append('erro')
        
df_normal['normalidade'] = normal_list
df_normal

,columns,normalidade
2,acres92,Normal
3,acres87,Normal
4,acres82,Normal


In [14]:
# nome das colunas que passaram no teste da normalidade
list_col = df_normal.query("normalidade == 'Normal'")['columns'].unique().tolist()

# Imputação com a mediana para variáveis com alta proporção de dados ausentes
for col in list_col:
    df[col].fillna(df[col].median(), inplace=True)

# Amostragem

## Amostragem Aleatória Simples sem reposição

Estimar: <br>
(1) A area total dedicada a fazendas em 1992,<br>
(2) A area média dedicada a fazendas em 1992,<br>
(3) A proporção de condados com menos de 250.000 acres em fazendas em 1992,<br>
(4) O total de condados com menos de 250.000 acres em fazendas em 1992.<br>

Para cada tipo de estimação, obter:<br>
(1) Obtenha as estimativas, erro padrão e intervalo da confiança para cada um dos planos amostrais;<br>
(2) Faça uma análise comparativa dos resultados no item anterior, em termos do viés, erro quadrático médio e coeficiente de variação<br>
(3) Considere uma estimativa da razão e de regressão da  ́area total (media) dedicada a fazendas em 1992, considerando como variável auxiliar a ́area total dedicada a fazendas em 1982 (acres82) e a área total dedicada a fazendas em 1987 (acres87).<br>
(4) Compare os resultados do item anterior em termos de precisão e variabilidade relativa.

In [15]:
def calcular_erro_padrao(amostra):
    # Tamanho da amostra (n)
    n = len(amostra)

    # Variância amostral
    variancia_amostral = statistics.variance(amostra)

    # Desvio padrão da amostra (s)
    desvio_padrao_amostral = math.sqrt(variancia_amostral)

    # Erro padrão (SE)
    erro_padrao = desvio_padrao_amostral / math.sqrt(n)

    return erro_padrao

In [16]:
def calcular_intervalo_confianca(amostra, nivel_confianca=0.95):
    # Remover valores ausentes (NaN) da amostra
    amostra = [x for x in amostra if not math.isnan(x)]

    # Tamanho da amostra (n)
    n = len(amostra)

    # Média amostral (estimativa da área média das fazendas)
    media_amostral = sum(amostra) / n

    # Desvio padrão da amostra
    desvio_padrao_amostral = statistics.stdev(amostra)

    # Graus de liberdade
    graus_liberdade = n - 1

    # Valor crítico t para o nível de confiança desejado
    valor_critico_t = stats.t.ppf((1 + nivel_confianca) / 2, graus_liberdade)

    # Erro padrão (SE)
    erro_padrao = desvio_padrao_amostral / math.sqrt(n)

    # Intervalo de confiança
    limite_inferior = media_amostral - valor_critico_t * erro_padrao
    limite_superior = media_amostral + valor_critico_t * erro_padrao

    return limite_inferior, limite_superior

#### (1) A area total dedicada a fazendas em 1992,<br>

In [17]:
# amostrar 500 linhas aleatórias do dataframe
df_aas = df.sample(n=500, random_state=SEED)

Com relação a $AAS_s$, um estimador não viciado do total populacional é
$$T(S) = \frac{N}{n}t(s)$$

In [18]:
# estimar área total dedicada a fazendas em 1992
area_total = df_aas['acres92'].sum()
area_total_estimada = area_total*3078/500
print(f'Estimação da área total em 1992: {area_total_estimada:.2f} acres')

# estimar erro padrão da área total
erro_padrao = calcular_erro_padrao(df_aas['acres92'])
print(f'Erro padrão da estimativa: {erro_padrao:.2f} acres')

# estimar intervalo de confiança da área total
limite_inferior, limite_superior = calcular_intervalo_confianca(df_aas['acres92'])
print(f'Intervalo de confiança: {limite_inferior:.2f} a {limite_superior:.2f} acres')

Estimação da área total em 1992: 1026864895.43 acres
Erro padrão da estimativa: 21534.51 acres
Intervalo de confiança: 291304.85 a 375923.81 acres


Para $AAA_S$, a média amostral
$$\bar{y} = \frac{1}{n}\sum_{i \in s}^nY_i = \frac{t(s)}{n}$$
é um estimador não viciado da média populacional

#### (2) A area média dedicada a fazendas em 1992,<br>

In [19]:
# estimar área média dedicada a fazendas em 1992
n=500 # amostra

n_farms = df_aas['farms92'].count()

# divide a area_total pelo n_farms
area_media = area_total / n_farms
print(f'Estimação da área média em 1992: {area_media:.2f} acres')

# estimar erro padrão da área média
erro_padrao = calcular_erro_padrao(df_aas['acres92'] / df_aas['farms92'])
print(f'Erro padrão da estimativa: {erro_padrao:.2f} acres')

# estimar intervalo de confiança da área média
limite_inferior, limite_superior = calcular_intervalo_confianca(df_aas['acres92'] / df_aas['farms92'])
print(f'Intervalo de confiança: {limite_inferior:.2f} a {limite_superior:.2f} acres')

Estimação da área média em 1992: 333614.33 acres
Erro padrão da estimativa: 405.28 acres
Intervalo de confiança: 543.28 a 2135.80 acres


#### (3) A proporção de condados com menos de 250.000 acres em fazendas em 1992,<br>

In [34]:
# filtrar county por acres92 < 250000 e pegar proporção
df_aas_250mil = df_aas[df_aas['acres92'] < 250000]
proporcao = len(df_aas_250mil) / n  # n=500
print(f'Proporção de fazendas com área < 250000 acres: {proporcao:.2f}')  # 0.58


amostra = df_aas['acres92'] < 250000
sucesso = sum(amostra)
proporcao_amostral = sucesso / len(amostra)


# calcular erro padrao
erro_padrao_prop = calcular_erro_padrao(amostra)
print(f'Erro padrão da estimativa: {erro_padrao:.3f}')


# Intervalo de confiança
confianca = 0.95
intervalo_confianca_prop = proportion_confint(sucesso, n, alpha=1-confianca, method='wilson')
print(f'Intervalo de confiança: {intervalo_confianca_prop[0]:.2f} a {intervalo_confianca_prop[1]:.2f}')

Proporção de fazendas com área < 250000 acres: 0.58
Erro padrão da estimativa: 0.022
Intervalo de confiança: 0.54 a 0.63


#### (4) O total de condados com menos de 250.000 acres em fazendas em 1992.<br>

In [56]:
# condados com menos de 250 mil acres em 1992
df_250mil = df_aas[df_aas['acres92'] < 250000]

# contar condados com menos de 250 mil acres em 1992
n_250 = len(df_250mil)
# printa
print(f'Número de condados com menos de 250 mil acres em 1992 na amostra: {n_250}')

Número de condados com menos de 250 mil acres em 1992 na amostra: 292


In [55]:
n = len(df_aas)
N = len(df)
# pegar proporção
proporcao = n_250 / n
# multiplica proporcao para estimar total
total_estimado = proporcao * N
print(f'Estimação do número de condados com menos de 250 mil acres em 1992: {total_estimado:.0f}')

Estimação do número de condados com menos de 250 mil acres em 1992: 1798


In [57]:
# # total de county do dataset
# N = len(df)

# # total de county com menos de 250 mil acres em 1992
# county_250acres = proporcao * N 
# #printa
# print(f'Estimativa do total de county com menos de 250 mil acres em 1992: {county_250acres:.0f}')

# erro padrão da estimativa
erro_padrao = erro_padrao_prop * N
print(f'Erro padrão da estimativa: {erro_padrao:.0f}')

# intervalo de confiança
limite_inferior, limite_superior = intervalo_confianca_prop[0] * N, intervalo_confianca_prop[1] * N
print(f'Intervalo de confiança: {limite_inferior:.0f} a {limite_superior:.0f}')

Erro padrão da estimativa: 68
Intervalo de confiança: 1663 a 1928


## Amostragem aleatória Estratificado por região (region) com alocação proporcional

#### (1) A area total dedicada a fazendas em 1992,<br>

In [59]:
# Quantidade de condados por região:
condados_por_regiao = df.groupby(['region']).size().to_dict()

# Porcentagem por região
porcentagem_por_regiao = {regiao: quant/3078 for regiao, quant in condados_por_regiao.items()}

# Tamanho da amostra por região:
amostra_por_regiao = {regiao: round(quant*500) for regiao, quant in porcentagem_por_regiao.items()}
amostra_por_regiao

{'NC': 171, 'NE': 36, 'S': 224, 'W': 69}

In [60]:
# Usando AASs para obter uma amostra proporcional em cada estrato (região)
amostra = pd.DataFrame()
for regiao, quantidade in amostra_por_regiao.items():
    amostra_regiao = df[df['region'] == regiao]
    amostra = pd.concat([amostra, amostra_regiao.sample(quantidade, random_state=SEED)])
amostra

,county,state,acres92,acres87,acres82,farms92,farms87,farms82,largef92,largef87,largef82,smallf92,smallf87,smallf82,region
829,OWEN COUNTY,IN,113129.0,115428.0,118596.0,622,665,748,14,14,7,17,24,30,NC
2972,ONEIDA COUNTY,WI,31777.0,44389.0,41112.0,99,106,117,7,11,12,5,5,5,NC
2032,CUYAHOGA COUNTY,OH,4060.0,5563.0,8854.0,133,150,193,0,0,0,68,71,81,NC
2952,GREEN LAKE COUNTY,WI,163145.0,161827.0,164188.0,705,711,705,11,9,9,28,34,17,NC
613,VAN BUREN COUNTY,IA,241422.0,244939.0,270672.0,752,783,883,35,33,33,35,37,46,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,KOOTENAI COUNTY,ID,131281.0,170739.0,199576.0,541,611,615,29,49,45,69,69,71,W
2898,DOUGLAS COUNTY,WA,918033.0,987885.0,970528.0,888,948,937,202,236,244,277,263,261,W
1924,RIO ARRIBA COUNTY,NM,1552865.0,1490672.0,1490281.0,964,936,825,140,125,126,226,205,155,W
185,MENDOCINO COUNTY,CA,725118.0,777264.0,782999.0,1088,1067,1084,118,128,132,161,132,155,W


In [61]:
# Calculando média de cada estrato
medias_estratos = amostra.groupby(['region'])['acres92'].mean().to_dict()

# Estimando a área total:
area_total_estimada = sum([media*condados_por_regiao.get(regiao) for regiao, media in medias_estratos.items()])

# Estimando a média populacional
media_pop_estimada = area_total_estimada/3078

print(f'Área total estimada = {area_total_estimada:.2f}')

Área total estimada = 964194037.98


#### (2) A area média dedicada a fazendas em 1992,<br>

Sabemos que para AASs, a média amostral $$\bar{y} = \frac{1}{n}\sum_{i \in s}^{n}Y_i$$ é um estimador não viesado para a média do estrato.\
Além disso, o estimador $$T_{es} = \sum_{h=1}^{H}N_h\hat{\mu}_h$$ é não viesado para o total populacional $\tau$

In [62]:
print(f'Área média estimada = {media_pop_estimada:.2f}')

Área média estimada = 313253.42


#### (3) A proporção de condados com menos de 250.000 acres em fazendas em 1992,<br>

Dada uma amostra $s_h$ de tamanho $n_h$ selecionada segundo a AASs no estrato h, pode-se então definir para P o estimador
$$\hat{P}_{es} = \sum_{h=1}^H W_h\frac{T_h}{n_h}$$
Onde $T_h$ é o numero de elementos na amostra que possuem a caracteristica no estrato h

In [64]:
# Quantidade de condados com menos de 250.000 acres de fazendo em 92 para cada estrato:
menor_250 = {}
for regiao in amostra['region'].unique():
    qnt = amostra[(amostra['region'] == regiao) & (amostra['acres92'] < 250000)].shape[0]
    menor_250[regiao] = qnt
menor_250

{'NC': 74, 'NE': 36, 'S': 170, 'W': 21}

In [65]:
Pes = sum([porcentagem_por_regiao.get(regiao)*menor_250.get(regiao)/amostra_por_regiao.get(regiao) for regiao in amostra['region'].unique()])
print(f'Proporção de condados com menos de 250.000 acres em fazendas: {Pes:.2%}')

Proporção de condados com menos de 250.000 acres em fazendas: 60.21%


#### (4) O total de condados com menos de 250.000 acres em fazendas em 1992.<br>

In [66]:
print(f'Total de condados com menos de 250.000 acres em fazendas: {round(Pes*3078)}')

Total de condados com menos de 250.000 acres em fazendas: 1853


## Amostragem Aleatória por conglomerados usando o estado (state) como a variável de cluster,